In [1]:
import pynvml

def list_gpus_memory():
    pynvml.nvmlInit()
    device_count = pynvml.nvmlDeviceGetCount()

    for i in range(device_count):
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        name = pynvml.nvmlDeviceGetName(handle)
        print(f"GPU {i}: {name}")
        print(f"  Total Memory: {info.total / (1024**2):.2f} MB")
        print(f"  Free Memory: {info.free / (1024**2):.2f} MB")
        print(f"  Used Memory: {info.used / (1024**2):.2f} MB")
        print()
    pynvml.nvmlShutdown()

if __name__ == "__main__":
    list_gpus_memory()

GPU 0: NVIDIA RTX A5000
  Total Memory: 24564.00 MB
  Free Memory: 20830.12 MB
  Used Memory: 3733.88 MB

GPU 1: NVIDIA RTX A5000
  Total Memory: 24564.00 MB
  Free Memory: 522.12 MB
  Used Memory: 24041.88 MB

GPU 2: Quadro RTX 5000
  Total Memory: 16384.00 MB
  Free Memory: 16114.38 MB
  Used Memory: 269.62 MB

GPU 3: Quadro RTX 5000
  Total Memory: 16384.00 MB
  Free Memory: 15700.38 MB
  Used Memory: 683.62 MB



In [2]:
import sys
import os
import numpy as np
import torch
import yaml
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import random

def set_seed(seed):
    """
    Set the seed for generating random numbers.

    Args:
        seed (int): The seed value to use.
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True  # for reproducibility
    torch.backends.cudnn.benchmark = False  # for reproducibility

# set seed
set_seed(42)
gpu_idx = 0
device = torch.device(f"cuda:{gpu_idx}") if torch.cuda.is_available() else torch.device("cpu")
torch.cuda.set_device(gpu_idx)
print("Device:", device)

Device: cuda:0


In [3]:
import pickle

# Open the file containing the pickled data
with open('dataset ml2 max depth.pkl', 'rb') as file:
    # Load data from file
    dataset = pickle.load(file)

In [4]:
for key in dataset:
    print(key, type(dataset[key]))

dataset_max_depth <class 'list'>
debit <class 'list'>
max_depth <class 'list'>


In [5]:
X = np.array(dataset['debit'])
y = np.array(dataset['max_depth'])
y[np.isnan(y)] = 0

In [6]:
X.shape, y.shape

((37, 72), (37, 3078, 2019))

In [6]:
kasus_test = [1,4,7,8,21,25,30]
kasus_train = [i for i in range(len(X)) if i not in kasus_test]

X_train, X_val  = X[kasus_train], X[kasus_test]
y_train, y_val = y[kasus_train], y[kasus_test]

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(30, 72) (7, 72) (30, 3078, 2019) (7, 3078, 2019)


In [10]:
data = {}
for n,i in enumerate(kasus_test):
    data[f'Kasus {i}'] = X_val[n]

In [13]:
import pickle

# Dump the object into a pickle file
with open('Kasus Validasi ML2.pkl', 'wb') as file:
    pickle.dump(data, file)

In [8]:
#preprocess data input
X_train = torch.tensor(X_train, dtype = torch.float32)
X_val = torch.tensor(X_val, dtype = torch.float32)

y_train = torch.tensor(y_train, dtype = torch.float32)
y_val = torch.tensor(y_val, dtype = torch.float32)

n_train,width,height = y_train.shape

In [9]:
def pre_process_data_for_fully_connected(X_train, y_train, X_val, y_val):
    """
    function to process data ready for fully connected model
    """
    n_train = len(X_train)
    n_val = len(X_val)
    X_train = X_train.view(n_train, -1)
    y_train = y_train.view(n_train, -1)
    X_val = X_val.view(n_val, -1)
    y_val = y_val.view(n_val, -1)
    return X_train, y_train, X_val, y_val

In [10]:
X_train, y_train, X_val, y_val = pre_process_data_for_fully_connected(X_train=X_train, y_train=y_train, 
                                                                      X_val=X_val, y_val=y_val)

In [11]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

torch.Size([30, 72]) torch.Size([7, 72]) torch.Size([30, 6214482]) torch.Size([7, 6214482])


In [12]:
def pre_process_data_for_CNN_SBKabir(X_train, y_train, X_val, y_val):
    """
    Function to proses data ready for CNN SBKabir
    """
    n_train, features = X_train.shape
    n_val, features = X_val.shape

    X_train = X_train.view(n_train, features, 1)
    X_val = X_val.view(n_val, features,1)
    
    return X_train, y_train, X_val, y_val

In [13]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def create_dataloader(X, y, batch_size, shuffle=False):
    dataset = CustomDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader
    
# # Data loaders
# batch_size = 2

# train_loader_model1 = create_dataloader(X=X_train, y=y_train, batch_size=batch_size, shuffle=True)
# val_loader_model1 = create_dataloader(X=X_val, y=y_val, batch_size=batch_size, shuffle=False)

In [14]:
def train(model, train_loader, val_loader, optimizer, criterion, epochs, device):
    model.train()
    train_error, ssim_val, mse_val = [], [], []
    for epoch in range(epochs):
        for X,y in train_loader:
            X,y = X.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output,y)
            loss.backward()
            optimizer.step()
        actual, predicted = evaluate_model(model, val_loader)
        ssim_values, mse_values = evaluate_flood_depth(actual, predicted)
        train_error.append(loss.item()); ssim_val.append(np.mean(ssim_values)); mse_val.append(np.mean(mse_values))
        print(f"Epoch {epoch+1} Train MSE  is {loss.item()}, Val MSE {np.mean(mse_values)}, Val SSIM {np.mean(ssim_values)}")
    return  model, train_error, ssim_val, mse_val

### Evaluation Function

In [15]:
from evaluation import calculate_ssim,calculate_mse,evaluate_flood_depth,evaluate_model

### Model CNN SBKabir

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim

class CNNModelBN(nn.Module):
    def __init__(self, steps, features, outputs):
        super(CNNModelBN, self).__init__()
        
        # Adjust in_channels to 72 based on the provided weight shape
        self.conv1 = nn.Conv1d(in_channels=features, out_channels=64, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(64)
        
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=256, kernel_size=1)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.flatten = nn.Flatten()
        
        # Adjust the input dimension of fc1 to 256 based on the provided weight shape
        self.fc1 = nn.Linear(256 * steps, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(256, 256)
        self.bn4 = nn.BatchNorm1d(256)
        self.dropout2 = nn.Dropout(0.2)
        
        self.fc3 = nn.Linear(256, 64)
        self.bn5 = nn.BatchNorm1d(64)
        
        # Adjust fc4 to match the output dimensions of your provided weight shape
        self.fc4 = nn.Linear(64, outputs)
        
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.dropout1(x)
        
        x = self.fc2(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.dropout2(x)
        
        x = self.fc3(x)
        x = self.bn5(x)
        x = self.relu(x)
        
        x = self.fc4(x)
        return x

In [26]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

torch.Size([30, 72, 1]) torch.Size([7, 72, 1]) torch.Size([30, 6214482]) torch.Size([7, 6214482])


In [22]:
X_train, y_train, X_val, y_val = pre_process_data_for_CNN_SBKabir(X_train, y_train, X_val, y_val)

batch_size = 2

train_loader_model_kabir = create_dataloader(X=X_train, y=y_train, batch_size=batch_size, shuffle=True)
val_loader_model_kabir = create_dataloader(X=X_val, y=y_val, batch_size=batch_size, shuffle=False)

In [23]:
def create_model_ml2(input_size, output_size):
    steps = 1
    model = CNNModelBN(steps=steps, features=input_size, outputs=output_size)
    return model

def load_model_ml2(input_size, output_size, path_model):
    model = create_model_ml2(input_size, output_size)
    # Load the saved weights into the model
    model.load_state_dict(torch.load(path_model))
    model.eval()
    print("Successfully  loaded ml2")
    return model

In [24]:
from torchsummary import summary
#_, width, height = y_train.shape
output_size = int(width * height)
features = 72
steps = 1

#model = CNNModelBN(steps, features, output_size).to(device)
path_pretrained = "Improved CNN Kabir ML2 100 epoch 37 kasus.pth"
model = load_model_ml2(input_size=features, output_size=output_size, path_model=path_pretrained)
model.to(device)
# define hyperparameter
lr = 0.001
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr)
epochs = 2

# Summarize the model
summary(model, input_size=(72, 1))

Successfully  loaded ml2
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1                [-1, 64, 1]           4,672
       BatchNorm1d-2                [-1, 64, 1]             128
              ReLU-3                [-1, 64, 1]               0
            Conv1d-4               [-1, 256, 1]          16,640
       BatchNorm1d-5               [-1, 256, 1]             512
              ReLU-6               [-1, 256, 1]               0
           Flatten-7                  [-1, 256]               0
            Linear-8                  [-1, 256]          65,792
       BatchNorm1d-9                  [-1, 256]             512
             ReLU-10                  [-1, 256]               0
          Dropout-11                  [-1, 256]               0
           Linear-12                  [-1, 256]          65,792
      BatchNorm1d-13                  [-1, 256]             512
             R

In [25]:
model, train_error, ssim_val, mse_val = train(model=model, train_loader=train_loader_model_kabir,val_loader= val_loader_model_kabir, optimizer=optimizer,
                            criterion=criterion, epochs = epochs, device = device)

Epoch 1 Train MSE  is 0.07964055240154266, Val MSE 0.034770760319424235, Val SSIM 0.9060371971761334
Epoch 2 Train MSE  is 0.025861844420433044, Val MSE 0.02361006697056631, Val SSIM 0.8700108657885919


In [ ]:
# # Save the model state
# torch.save(model.state_dict(), 'CNN Kabir 100 epoch run.pth')

In [ ]:
# plot_training_error(mse_val, filename ="validation_error_mse_cnn_kabir.png")

In [ ]:
# y_val = y_val.view(-1, width, height)
# y_val = y_val.numpy()

# with torch.no_grad():
#     predicted_val = model(X_val.to(device))
#     predicted_val = predicted_val.view(-1, width,height)
#     predicted_val = predicted_val.to("cpu").numpy()

In [ ]:
# from utils import convert_array_to_tif


# filename = ["Predicted kasus 6.tif", "Predicted kasus 7.tif"]

# for n,file in enumerate(filename):
#     convert_array_to_tif(predicted_val[n], filename[n])

In [ ]:
# n = 0
# compare_images(n=n, predicted= predicted_val[n], ground_truth=y_val[n], save= True, title='Kabir Comparison of Predicted and Ground Truth Images')

### Simple Model

In [16]:
class SimpleFC(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleFC, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        return x

In [17]:
import numpy as np

# Create an array of 100 random values
random_values = np.random.rand(1,100)
random_values.shape

(1, 100)

In [18]:
144 * 114 

16416

In [19]:
#define modele
from torchsummary import summary
input_size = 72
output_size = width * height
model = SimpleFC(input_size = input_size, output_size = output_size).to(device)
# define hyperparameter
lr = 0.001
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr)
epochs = 200
# Summarize the model
summary(model, input_size=(1,72))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 64]           4,672
              ReLU-2                [-1, 1, 64]               0
            Linear-3                [-1, 1, 64]           4,160
              ReLU-4                [-1, 1, 64]               0
            Linear-5           [-1, 1, 6214482]     403,941,330
              ReLU-6           [-1, 1, 6214482]               0
Total params: 403,950,162
Trainable params: 403,950,162
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 94.83
Params size (MB): 1540.95
Estimated Total Size (MB): 1635.78
----------------------------------------------------------------


In [20]:
model, train_error, ssim_val, mse_val = train(model=model, train_loader=train_loader_model1,val_loader= val_loader_model1, optimizer=optimizer,
                            criterion=criterion, epochs = epochs, device = device)

NameError: name 'train_loader_model1' is not defined

In [ ]:
# # Save the model state
# torch.save(model.state_dict(), 'Simple FC ML2 100 epoch run 2.pth')

#load model 
# Load the saved model weights
model.load_state_dict(torch.load('Simple FC ML2 100 epoch run 2.pth'))

In [ ]:
import matplotlib.pyplot as plt

def plot_training_error(train_errors, title='Training Error over Epochs', filename='training_error_plot.png'):
    """
    Plot the training errors over epochs and save the plot to a file.

    Parameters:
    - train_errors: A list of training error values.
    - title: A string to title the plot (optional).
    - filename: The filename to save the plot to (optional).
    """
    # Number of epochs is the length of the train_errors list
    epochs = range(1, len(train_errors) + 1)

    # Create the plot
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_errors, label='Training Error', linestyle='-')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.grid(True)
    plt.legend()

    # Save the plot
    plt.savefig(filename)
    plt.close()  # Close the plot window to free memory

    print(f"Plot saved as {filename}")

#plot_training_error(train_error, filename ="training_error_mse_plot_simple_fc_run2.png")

In [ ]:
with torch.no_grad():
    predicted_val = model(X_val.to(device))
    predicted_val = predicted_val.view(-1, width,height)
    predicted_val = predicted_val.to("cpu").numpy()

# y_val = y_val.view(-1, width, height)
# y_val = y_val.numpy()

In [ ]:
from utils import convert_array_to_tif


filename = ["FC Sederhana Predicted kasus 6.tif", "FC Sederhana Predicted kasus 7.tif"]

for n,file in enumerate(filename):
    convert_array_to_tif(predicted_val[n], filename[n])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def compare_images(n, predicted, ground_truth,save, title='Comparison of Predicted and Ground Truth Images'):
    """
    This function takes two images, 'predicted' and 'ground_truth', and plots them side by side with a color bar indicating the range of pixel values.
    
    Parameters:
    - predicted: The predicted image as a numpy array.
    - ground_truth: The ground truth image as a numpy array.
    - title: A string that represents the title of the plot.
    """
    fig, ax = plt.subplots(1, 2, figsize=(12, 8))
    
    # Display predicted image
    img1 = ax[0].imshow(predicted, aspect='equal', cmap = "gray_r")
    ax[0].title.set_text('Predicted Image')
    ax[0].axis('off')
    
    # Display ground truth image
    img2 = ax[1].imshow(ground_truth, aspect='equal', cmap = "gray_r")
    ax[1].title.set_text('Ground Truth Image')
    ax[1].axis('off')
    
    # Add a color bar
    fig.colorbar(img1, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
    
    # Set the main title
    plt.suptitle(title)
    
    #plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to make room for the main title
    if save:
        plt.savefig(f"900Reverse Comparison of Predicted and actual flood depth with bar {n}.png")
    plt.show()

In [ ]:
n = 1
compare_images(n=n, predicted= predicted_val[n], ground_truth=y_val[n], save= True, title='Comparison of Predicted and Ground Truth Images')